# Generating text in Rumi-style, by training NLP model on Rumi poetry dataset

### Auto-generated cell from Kaggle

In [63]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import tensorflow as tf # Deep Learning framework
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

data_dir = "./datasets/"

import os
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./datasets/rumi_poetry.xlsx


In [66]:
# Getting source file name / path
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
source_file = filenames[0]
source_file

'./datasets/rumi_poetry.xlsx'

In [68]:
# Creating data source DataFrame
source_df = pd.read_excel(source_file)
source_df

,Title,Poem
0,Unknown Existence,unknown existence\nundiscovered beauty\nthat's...
1,In This Earth,in this earth\nin this earth\nin this immacula...
2,All The Precious Words,all the precious words\nyou and i have exchang...
3,The Sweetheart,the sweetheart\nwho is blocking my sleep\ndema...
4,I Know of a Path in Your Heart,i know of a path in your heart\nthat merges wi...
...,...,...
83,Come,come \nwhoever you are\ncome\ncome again\neven...
84,I Am No Lion,i am no lion \nto overpower my enemies\nwinnin...
85,Restless,restless\nnow i go to the door\nnow i go on th...
86,All My Friends,all my friends\ndeparted like dreams\nleft alo...


Original TensorFlow documentation sample on "Text Generation with an RNN":
https://www.tensorflow.org/text/tutorials/text_generation

In [19]:
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)

In [20]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [21]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [22]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [23]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [24]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [25]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [26]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [27]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [28]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [29]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [30]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [31]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [32]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [33]:
seq_length = 100

In [34]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [35]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [36]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [37]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [38]:
dataset = sequences.map(split_input_target)

In [39]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [40]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [41]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [42]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [43]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [44]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [45]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [46]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [47]:
sampled_indices

array([36, 64, 44, 32, 32, 22, 11, 53, 18, 11, 60, 64, 17, 32, 47, 55, 38,
       20, 17, 57, 11, 26, 63, 10, 58,  5, 32, 43, 35, 43, 12, 23, 23, 40,
        6, 12, 22, 55,  7, 29, 64,  7, 37, 58, 10, 18, 10, 17, 34,  8, 27,
       30,  7, 52, 51,  2, 50,  7, 65,  3, 38, 44, 14, 18, 40, 52, 16, 19,
       21, 41, 36, 60,  1,  7,  6, 61, 12, 63,  3, 49, 11, 25, 63, 45, 53,
       53,  2, 12, 61, 60, 64, 45, 60,  7, 40, 30,  0, 57, 47,  4],
      dtype=int64)

In [49]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'uncle, do you know the cause?\n\nMONTAGUE:\nI neither know it nor can learn of him.\n\nBENVOLIO:\nHave you'

Next Char Predictions:
 b"WyeSSI:nE:uyDShpYGDr:Mx3s&SdVd;JJa';Ip,Py,Xs3E3DU-NQ,ml k,z!YeAEamCFHbWu\n,'v;x!j:Lxfnn ;vuyfu,aQ[UNK]rh$"


In [50]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [51]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1902356, shape=(), dtype=float32)


In [52]:
tf.exp(example_batch_mean_loss).numpy()

66.038345

In [53]:
model.compile(optimizer='adam', loss=loss)

In [54]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [55]:
EPOCHS = 20

In [56]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 733s 4s/step - loss: 2.7401
Epoch 2/20
172/172 [==============================] - 6898s 40s/step - loss: 2.0040
Epoch 3/20
172/172 [==============================] - 470s 3s/step - loss: 1.7245
Epoch 4/20
172/172 [==============================] - 588s 3s/step - loss: 1.5591
Epoch 5/20
172/172 [==============================] - 707s 4s/step - loss: 1.4575
Epoch 6/20
172/172 [==============================] - 718s 4s/step - loss: 1.3867
Epoch 7/20
172/172 [==============================] - 583s 3s/step - loss: 1.3333
Epoch 8/20
172/172 [==============================] - 311s 2s/step - loss: 1.2882
Epoch 9/20
172/172 [==============================] - 324s 2s/step - loss: 1.2466
Epoch 10/20
172/172 [==============================] - 325s 2s/step - loss: 1.2066
Epoch 11/20
172/172 [==============================] - 325s 2s/step - loss: 1.1671
Epoch 12/20
172/172 [==============================] - 330s 2s/step - loss: 1.1261
Epoch 13/20

In [57]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [58]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [59]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
The world yield me; go and he smullows:
This side is bloody disingrawel;
But spoil you seek to many greatest strength;
But show'd me hate: at laugh at thee?

FRIAR LAURENCE:
I callument she lies yet.

BRUTUS:
Mark'd do yourself these words; the unjusticient malice
may's absence more a fool thou art a
proclained blood, none fit out with victorious wreads;
Resign thy catter's worship, would not but
the shape, nor waults upon the feather and that
Romeo.

MARIANA:
Because six on that name; yea, one word I am tail
Ty to his captain through a thorny wood,
Whilst thou not slander with a habelest
slepy.

Servant:
God save him, Angelo!

ARIEL:
The god of sailors shall were butts, being so hething.

EDWARD:
O pity, sir, that you may.
Can she had come to lay myself as you.

DUKE VINCENTIO:
Be gone:
Tyrant your state great rootent. I will but
With all your painings. Romeo hast not mack'd thee
From those which I must give.

ELWO:
True, I think, if you be he not.

DUKE VINCENTIO:
For every da

In [60]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThis doth not string from thee: thou do abott\nMine easier the dee--trow down,\nMust by the rock of strange, no lady-pain,\nBecause since with Juliet is prey, stands.\n\nHERMIONE:\nNo, I can read.\n\nSecond Citizen:\nYour will, sir, what dost thou not shame,\nWhich being castain, like lambs; he begs'd to Bolingbroke\nA nabil up to hear at his command.\n\nPOLIXENES:\nAy, but good morrow, gentle met me.\n\nPOMPEY:\nI told you for your day, and all report that good line\nBeing queen, your subjects. You, she wills,\nI would not, that's a tale stomens.\n\nESCALUS:\nIt is the enfolding I swore oracle\nI mean to do so; you show your flesh and breath\nAs when they shall succorder them as we place.\n\nPOLIXENES:\nO, 'tis a good speedy well! Come along?\n\nDUKE OF YORK:\nIt would inconstant thee, from The relatit ere\nA mile of such a name of moen inclence\nBefore he show'd our conquest did.\n\nNurse:\nThis is a heavy complace: come I may not say no.\n\nVOLUMNIA:\nThou hast

In [61]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step\assets


INFO:tensorflow:Assets written to: one_step\assets


In [62]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The doom of this is sin under thy tale
Again to prison. Good! get thee gone,
Which seizest thou fox
